In [20]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import pickle
from utils import *
import time

from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode
import zlib

import warnings
warnings.filterwarnings('ignore')

img = cv2.imread('beer.png')

In [2]:
root = '/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/cortices/'

R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

C = Cortex(gpu=False)
C.loadLocs(root+'50k_Lloc_tight.pkl', root+'50k_Rloc_tight.pkl')
C.loadCoeffs(root+'50k_Lcoeff_tight.pkl', root+'50k_Rcoeff_tight.pkl')

In [18]:
def decode_data(data):
    encoded_V = data[0]
    code_dict = data[1]
    bits = data[2]
    
    V = huffman_decode(encoded_V, symbol_code_dict=code_dict, stream_length=bits)
    return V.reshape((50000, 3))

In [21]:
import zmq
context = zmq.Context()

print("Connecting to Client…")
client_socket = context.socket(zmq.PULL)
client_socket.bind("tcp://*:5555")

while True:
    data_type = client_socket.recv()
    
    ret, cort = False, False
    if data_type == b'Ret':
        ret = True
    elif data_type == b'Cort':
        cort = True
   
    start = time.time()
    while True:
        data = client_socket.recv()
        if data == b'Done':
            break
        
        decompressed_data = data#zlib.decompress(data)
        image = pickle.loads(decompressed_data)

        if ret:
            V = decode_data(image)
            image = R.backproject_tight(V, (1080,1920,3), fixation)
        elif cort:
            V = decode_data(image)
            image = C.cort_img(V)
        
        cv2.imshow('Image', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    end = time.time()
    client_time = float(client_socket.recv())
    server_time = end-start
    
    cv2.destroyWindow('Image')
    cv2.waitKey(1)
    
    print("----------Report----------")
    print("Total time: %f"%(client_time+server_time))
    print("Client side total time: %f"%(client_time))
    print("Server side total time: %f"%(server_time))
    print()


Connecting to Client…
----------Report----------
Total time: 18.929569
Client side total time: 7.827518
Server side total time: 11.102051

----------Report----------
Total time: 16.896350
Client side total time: 6.911135
Server side total time: 9.985215



KeyboardInterrupt: 

In [ ]:
video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()

count = 1
while success:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if count ==1:
        print(img.shape)
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success, img = vidcap.read()
    count += 1

cv2.destroyWindow('Image')
cv2.waitKey(1)
vidcap.release()